# Task3 基于机器学习的文本分类
----
在上一章节，我们对赛题的数据进行了读取，并在末尾给出了两个小作业。如果你顺利完成了作业，那么你基本上对Python也比较熟悉了。在本章我们将使用传统机器学习算法来完成新闻分类的过程，将会结束到赛题的核心知识点。

## 基于机器学习的文本分类
----
在本章我们将开始使用机器学习模型来解决文本分类。机器学习发展比较广，且包括多个分支，本章侧重使用传统机器学习，从下一章开始是基于深度学习的文本分类。

## 学习目标
----
- 学会TF-IDF的原理和使用
- 使用sklearn的机器学习模型完成文本分类

## 机器学习模型
----
机器学习是对能通过经验自动改进的计算机算法的研究。机器学习通过历史数据训练出模型对应于人类对经验进行归纳的过程，机器学习利用模型对新数据进行预测对应于人类利用总结的规律对新问题进行预测的过程。

机器学习有很多种分支，对于学习者来说应该优先掌握机器学习算法的分类，然后再其中一种机器学习算法进行学习。由于机器学习算法的分支和细节实在是太多，所以如果你一开始就被细节迷住了眼，你就很难知道全局是什么情况的。

如果你是机器学习初学者，你应该知道如下的事情：

- 机器学习能解决一定的问题，但不能奢求机器学习是万能的；
- 机器学习算法有很多种，看具体问题需要什么，再来进行选择；
- 每种机器学习算法有一定的偏好，需要具体问题具体分析；

![](https://camo.githubusercontent.com/d8e9a12417a2a2a754a874af0ae163bb1bddbb0b/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303731343230333232333235332e6a7067)

## 文本表示方法 Part1
----
在机器学习算法的训练过程中，假设给定$ N $个样本，每个样本有$ M $个特征，这样组成了$N×M$的样本矩阵，然后完成算法的训练和预测。同样的在计算机视觉中可以将图片的像素看作特征，每张图片看作hight×width×3的特征图，一个三维的矩阵来进入计算机进行计算。

但是在自然语言领域，上述方法却不可行：文本是不定长度的。文本表示成计算机能够运算的数字或向量的方法一般称为词嵌入（Word Embedding）方法。词嵌入将不定长的文本转换到定长的空间内，是文本分类的第一步。

### One-hot
----
这里的One-hot与数据挖掘任务中的操作是一致的，即将每一个单词使用一个离散的向量表示。具体将每个字/词编码一个索引，然后根据索引进行赋值。
One-hot表示方法的例子如下：

```
句子1：我 爱 北 京 天 安 门
句子2：我 喜 欢 上 海
```

首先对所有句子的字进行索引，即将每个字确定一个编号：

In [2]:
{
    '我': 1, '爱': 2, '北': 3, '京': 4, '天': 5,
  '安': 6, '门': 7, '喜': 8, '欢': 9, '上': 10, '海': 11
}

{'我': 1,
 '爱': 2,
 '北': 3,
 '京': 4,
 '天': 5,
 '安': 6,
 '门': 7,
 '喜': 8,
 '欢': 9,
 '上': 10,
 '海': 11}

在这里共包括11个字，因此每个字可以转换为一个11维度稀疏向量：
```
我：[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
爱：[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
...
海：[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
```

### Bag of Words
----
Bag of Words（词袋表示），也称为Count Vectors，每个文档的字/词可以使用其出现次数来进行表示。

```
句子1：我 爱 北 京 天 安 门
句子2：我 喜 欢 上 海
```

直接统计每个字出现的次数，并进行赋值：
```
句子1：我 爱 北 京 天 安 门
转换为 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

句子2：我 喜 欢 上 海
转换为 [1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
```

在sklearn中可以直接CountVectorizer来实现这一步骤：

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]])

### N-gram
----
N-gram与Count Vectors类似，不过加入了相邻单词组合成为新的单词，并进行计数。

如果N取值为2(bigram)，则句子1和句子2就变为：

```
句子1：我爱 爱北 北京 京天 天安 安门
句子2：我喜 喜欢 欢上 上海
```

In [9]:
def n_gram(target, n):
  # 提取n个字，同时通过＃1开始的字移动
  return [ target[idx:idx + n] for idx in range(len(target) - n + 1)]

target1 = "我爱北京天安门"
target2 = "我喜欢上海"

# unigram
print("unigram:")
print(n_gram(target1, 1))
print(n_gram(target2, 1))
# bigram
print("bigram:")
print(n_gram(target1, 2))
print(n_gram(target2, 2))
# trigram
print("trigram:")
print(n_gram(target1, 3))
print(n_gram(target2, 3))

unigram:
['我', '爱', '北', '京', '天', '安', '门']
['我', '喜', '欢', '上', '海']
bigram:
['我爱', '爱北', '北京', '京天', '天安', '安门']
['我喜', '喜欢', '欢上', '上海']
trigram:
['我爱北', '爱北京', '北京天', '京天安', '天安门']
['我喜欢', '喜欢上', '欢上海']


### TF-IDF
----
TF-IDF 分数由两部分组成：第一部分是词语频率（Term Frequency），第二部分是逆文档频率（Inverse Document Frequency）。其中计算语料库中文档总数除以含有该词语的文档数量，然后再取对数就是逆文档频率。

```
TF(t)= 该词语在当前文档出现的次数 / 当前文档中词语的总数
IDF(t)= log_e（文档总数 / 出现该词语的文档总数）
```

## 基于机器学习的文本分类
----
接下来我们将对比不同文本表示算法的精度，通过本地构建验证集计算F1得分。

### Count Vectors + RidgeClassifier
----

In [10]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=15000)

vectorizer = CountVectorizer(max_features=3000)
train_test = vectorizer.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.6547367843040645


### TF-IDF + RidgeClassifier
----

In [11]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=15000)

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.8719098297954606


## 本章小结
----
本章我们介绍了基于机器学习的文本分类方法，并完成了两种方法的对比。

## 本章作业
----
- 尝试改变TF-IDF的参数，并验证精度


In [13]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=85000)#15000->85000

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=5000)#3000->5000
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.8882287312788192


参数修改后：由原来的**0.8719098297954606** ，上升到**0.8882287312788192**

In [16]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=850000)#15000->850000

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=50000)#3000->50000
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.9018369345026127


参数修改后：由原来的**0.8882287312788192** ，上升到**0.9018369345026127**

In [20]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=1050000)#15000->1050000

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=80000)#3000->80000
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.9025402333065486


In [22]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=1050000)#15000->1050000

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=100000)#3000->100000
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.9025355728919735


参赛格式如下：

In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier

train_df = pd.read_csv('../input/train_set.csv', sep='\t')
test_df = pd.read_csv('../input/test_a.csv', sep='\t')

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=100000).fit(train_df['text'].iloc[:].values)
train_tfidf = tfidf.transform(train_df['text'].iloc[:].values)
test_tfidf = tfidf.transform(test_df['text'].iloc[:].values)

clf = RidgeClassifier()
clf.fit(train_tfidf,train_df['label'].iloc[:].values)

df = pd.DataFrame()
df['label'] = clf.predict(test_tfidf)
df.to_csv('submit.csv',index=None)

#### 我的成绩：0.9357

- 尝试使用其他机器学习模型，完成训练和验证

In [14]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=85000)#15000->85000

vectorizer = CountVectorizer(max_features=5000)#3000->5000
train_test = vectorizer.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.6601799764203219


参数修改后：由原来的**0.6547367843040645** ，上升到**0.6601799764203219**

In [17]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=850000)#15000->850000

vectorizer = CountVectorizer(max_features=50000)#3000->50000
train_test = vectorizer.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.6604850700662162


参数修改后：由原来的***0.6601799764203219***，上升到***0.6604850700662162***

In [21]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=1050000)#15000->1050000

vectorizer = CountVectorizer(max_features=80000)#3000->80000
train_test = vectorizer.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.6604546833580189


## 比较列表
----

| 模型 |次数| nrows参数 | max_features参数 | 得分 |增长趋势|最大值|
|:----|:----|:----|:----|:----|:----|:----|
|BOW|原始|15000|3000|0.6547367843040645|可能|X|
||第一次|85000|5000|0.6601799764203219|可能|X|
||第二次|850000|50000|0.6604850700662162|可能|V|
||第三次|1050000|80000|0.6604546833580189|不可能|X|
|TF-IDF|原始|15000|3000|0.8719098297954606|可能|X|
||第一次|85000|5000|0.8882287312788192|可能|X|
||第二次|850000|50000|0.9018369345026127|可能|X|
||第三次|1050000|80000|0.9025402333065486|可能|V|
||第四次|1050000|100000|0.9025355728919735|不可能|X|

**通过比较可以看出：TF-IDF通过修改参数提高精度方面，要优于BOW**

BOW：   调参达到最大分数0.6604850700662162 
TF-IDF：调参达到最大分数0.9025402333065486